In [1]:
import pandas as pd
import numpy as np
import glob, os

## 무 전처리

In [5]:
path = '../../files/도매시장/미나리.xlsx'

minari =  pd.read_excel(path)
minari.head()

,일자,도매시장,법인,부류,품목,출하지,물량(kg),금액(원)
0,20210427,서울가락,동화청과(주),엽경채류,미나리,경기도 안산시,1444.0,2981700
1,20210427,서울가락,동화청과(주),엽경채류,미나리,경기도 시흥시,6448.0,12769100
2,20210424,서울가락,(주)중앙청과,엽경채류,미나리,경기도 시흥시,5456.0,10322700
3,20210426,서울가락,동화청과(주),엽경채류,미나리,경기도 이천시,768.0,1569400
4,20210426,서울가락,동화청과(주),엽경채류,미나리,전라남도 나주시,2372.0,3762000


In [6]:
minari.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141515 entries, 0 to 141514
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   일자      141515 non-null  int64  
 1   도매시장    141515 non-null  object 
 2   법인      141515 non-null  object 
 3   부류      141515 non-null  object 
 4   품목      141515 non-null  object 
 5   출하지     141515 non-null  object 
 6   물량(kg)  141515 non-null  float64
 7   금액(원)   141515 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 8.6+ MB


In [7]:
minari.isna().sum()

일자        0
도매시장      0
법인        0
부류        0
품목        0
출하지       0
물량(kg)    0
금액(원)     0
dtype: int64

In [10]:
#법인 버리기
#minari.drop(['법인'], axis=1, inplace=True)

#날짜 바꾸기
minari['일자'] = pd.to_datetime(minari['일자'], format='%Y%m%d')
#일자 소팅
minari = minari.sort_values(by='일자')
minari.head()

,일자,도매시장,부류,품목,출하지,물량(kg),금액(원)
71636,2019-01-02,광주서부,엽경채류,미나리,광주광역시 광산구,60.0,363000
139860,2019-01-02,춘천,엽경채류,미나리,서울특별시,44.0,254960
111429,2019-01-02,창원팔용,엽경채류,미나리,경상남도 창원시 의창(성산)구,4.0,22000
78849,2019-01-02,전주,엽경채류,미나리,-,850.0,1274000
78946,2019-01-02,전주,엽경채류,미나리,-,3168.0,6028900


In [11]:
#도수분포?
print(minari['물량(kg)'].value_counts())
print(minari['금액(원)'].value_counts())

20.0      3978
40.0      3551
120.0     2846
80.0      2844
60.0      2797
          ... 
362.4        1
11.9         1
1853.0       1
6142.5       1
1013.2       1
Name: 물량(kg), Length: 6688, dtype: int64
120000     793
60000      737
300000     632
90000      554
240000     552
          ... 
57050        1
4591700      1
131158       1
7859900      1
40940        1
Name: 금액(원), Length: 40869, dtype: int64


In [22]:
len(minari)

141470

In [20]:
np.sum((minari['물량(kg)'] != 0) & (minari['금액(원)'] != 0))

141470

In [13]:
data_filter = (minari['물량(kg)'] != 0) & (minari['금액(원)'] != 0)
#약 45개 버림.#0값 찾기

minari = minari.loc[data_filter]

In [23]:
minari['출하지'] = minari['출하지'].str.split(" ").str[0]
minari['출하지'].unique()     # 수입산, 원양산, - : 제거 

array(['광주광역시', '서울특별시', '경상남도', '-', '전라북도', '울산', '광주', '전라남도', '경상북도',
       '경기도', '부산광역시', '강원도', '부산', '충청북도', '서울', '충청남도', '인천광역시',
       '울산광역시', '대구광역시', '대전광역시', '인천', '대전', '대구', '세종시', '제주도', '수입산'],
      dtype=object)

In [35]:
minari['출하지'].replace(['수입산', '원양산', '-', '서울', '인천', '부산', '울산', '광주', '대전','대구'],
                      [np.nan, np.nan, np.nan, '서울특별시', '인천광역시', '부산광역시', '울산광역시', '광주광역시', '대전광역시','대구광역시'],
                      inplace=True)
minari = minari.dropna(subset=['출하지'])
print(minari['출하지'].unique())
print(len(minari['출하지'].unique()))

['광주광역시' '서울특별시' '경상남도' '전라북도' '울산광역시' '전라남도' '경상북도' '경기도' '부산광역시' '강원도'
 '충청북도' '충청남도' '인천광역시' '대구광역시' '대전광역시' '세종시' '제주도']
17


In [36]:
minari['kg당 금액'] = 0
minari['kg당 금액'] = minari['금액(원)']/minari['물량(kg)']

<ipython-input-36-c310e27a887c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minari['kg당 금액'] = 0
<ipython-input-36-c310e27a887c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minari['kg당 금액'] = minari['금액(원)']/minari['물량(kg)']


In [38]:
minari

,일자,도매시장,부류,품목,출하지,물량(kg),금액(원),kg당 금액
71636,2019-01-02,광주서부,엽경채류,미나리,광주광역시,60.0,363000,6050.000000
139860,2019-01-02,춘천,엽경채류,미나리,서울특별시,44.0,254960,5794.545455
111429,2019-01-02,창원팔용,엽경채류,미나리,경상남도,4.0,22000,5500.000000
76662,2019-01-02,익산,엽경채류,미나리,전라북도,988.0,1886000,1908.906883
76043,2019-01-02,익산,엽경채류,미나리,전라북도,280.0,515000,1839.285714
...,...,...,...,...,...,...,...,...
109633,2021-10-02,대구북부,엽경채류,미나리,경상북도,23.0,59800,2600.000000
140384,2021-10-02,춘천,엽경채류,미나리,서울특별시,6.0,38040,6340.000000
109630,2021-10-02,대구북부,엽경채류,미나리,경상북도,20.0,100000,5000.000000
28011,2021-10-02,서울가락,엽경채류,미나리,경기도,247.5,945000,3818.181818


In [44]:
minari_mean = pd.DataFrame(minari.iloc[:, 7].groupby(minari['출하지']).agg('mean'))
minari_mean['kg당 금액'] = round(minari_mean['kg당 금액'], 0)
minari_mean

,kg당 금액
출하지,
강원도,3172.0
경기도,2854.0
경상남도,3345.0
경상북도,5577.0
광주광역시,2780.0
대구광역시,5886.0
대전광역시,2632.0
부산광역시,2887.0
서울특별시,3853.0


In [55]:
#반올림 적용
minari['kg당 금액']=round(minari['kg당 금액'], 0)

<ipython-input-55-6f3d26a2a257>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minari['kg당 금액']=round(minari['kg당 금액'], 0)


In [48]:
minari['일자']=minari['일자'].dt.strftime('%Y-%m-%d')
#뒤에 시간이 붙길래 적용함.

<ipython-input-48-d38adb52f241>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minari['일자']=minari['일자'].dt.strftime('%Y-%m-%d')


In [57]:
minari.to_csv("../../files/전처리/미나리.csv",index=False,encoding='utf-8-sig')